In [ ]:
# devtools::install_github("cellgeni/sceasy")

In [1]:
library(scater)
library(scDblFinder)
library(BiocParallel)
source("../tools/formating/formating.R")

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    

In [2]:
# path = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"
path = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"
srat <- LoadSeurat(path)
srat

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
X -> counts; scale.data -> data

Adding AnnData embeddings to Seurat assays

Adding AnnData embeddings to Seurat assays

Adding AnnData embeddings to Seurat assays



An object of class Seurat 
23341 features across 2392 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data
 3 dimensional reductions calculated: pca, tsne, umap

In [3]:
# SaveH5Seurat(srat, filename = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat", overwrite = TRUE, verbose = FALSE)
saveRDS(object = srat, file = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")

In [ ]:
robj <- readRDS("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")
robj

In [4]:
srat <- LoadSeurat("//ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds")
# srat <- LoadSeurat("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat")
srat

An object of class Seurat 
23341 features across 2392 samples within 1 assay 
Active assay: RNA (23341 features, 0 variable features)
 2 layers present: counts, data
 3 dimensional reductions calculated: pca, tsne, umap

In [ ]:
SaveH5Seurat(srat, filename = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat", overwrite = TRUE, verbose = FALSE)
srat <- LoadSeurat("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat")
srat

In [ ]:
srat <- tryCatch(
        LoadSeurat(path),
        error = function(e) {
            stop("The file format is not supported.")
            print(e)
        }
    )

In [ ]:
srat

In [ ]:
UMAPPlot(object = srat, label=T)

In [ ]:
srat@meta.data

In [ ]:
colnames(srat[['RNA']])

In [ ]:
srat[['pca']]

In [ ]:
srat <- Seurat::ReadH5AD(path)
srat

In [ ]:
adata <- LoadAnndata(path)
adata

In [ ]:
# Add dimension reduction
  embed_names <- unlist(adata$obsm_keys())
  if (length(embed_names) > 0) {
    embeddings <- sapply(embed_names, function(x) as.matrix(adata$obsm[[x]]), simplify = FALSE, USE.NAMES = TRUE)
    names(embeddings) <- embed_names
      for (name in embed_names) {
        rownames(embeddings[[name]]) <- colnames(srat[['RNA']])
      }
  }

In [ ]:
embeddings

In [ ]:
length(adata$obsm[['X_tsne']])

In [ ]:
adata$obsm[['X_tsne']]

In [ ]:
adata$layers

In [ ]:
adata$layers <- NULL
adata

In [ ]:
'' %in% names(adata$layers)

In [ ]:
names(adata$layers)

In [ ]:
for (layer in names(adata$layers)){
    print(layer)
  }

In [ ]:
is.null(names(adata$layers))

In [ ]:
names(adata$layers) == 'aaa'

In [ ]:
'scale.data' %in% names(adata$layers)

In [ ]:
adata$layers['scale.data']

In [ ]:
srat <- LoadSeurat("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat")
srat

In [ ]:
print(srat[['umap']])

In [ ]:
dims=1:10
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))

# PCA
# srat <- RunPCA(srat, features = VariableFeatures(srat), ndims.print = 6:10, nfeatures.print = 10)
srat <- RunPCA(srat, features = VariableFeatures(srat))

srat <- FindNeighbors(srat, dims=dims)
srat <- FindClusters(srat, resolution = 0.5)
srat

# TSNE
srat <- RunTSNE(srat, dims=dims)
# UMAP
srat <- RunUMAP(srat, dims=dims)

umap <- Embeddings(object = srat, reduction = "umap")

In [ ]:
embed_names <- unlist(adata$obsm_keys())
embed_names

In [ ]:
adata$obsm[['X_umap']]

In [ ]:
adata$obsm[['X_pca']]

In [ ]:
as.matrix(adata$obsm[['X_pca']])

In [ ]:
adata_Seurat <- CreateSeuratObject(counts = t(adata$X), meta.data = adata$obs)
adata_Seurat

In [ ]:
adata_Seurat$data

In [ ]:
if (reticulate::py_to_r(sp$issparse(adata$X))) {
      X <- Matrix::t(reticulate::py_to_r(sp$csc_matrix(adata$X)))
    } else {
      X <- t(reticulate::py_to_r(adata$X))
    }

In [ ]:
adata$X

In [ ]:
.obs2metadata <- function(obs_pd, assay = "RNA") {
  obs_df <- .regularise_df(obs_pd, drop_single_values=FALSE, drop_na_values=TRUE)
  colnames(obs_df) <- sub("n_counts", paste0("nCounts_", assay), colnames(obs_df))
  colnames(obs_df) <- sub("n_genes", paste0("nFeaturess_", assay), colnames(obs_df))
  return(obs_df)
}

#' Prepare feature metadata
#'
#' This function prepare feature metadata from AnnData.var
#'
#' @param var_pd Input AnnData.var dataframe
#'
#' @return AnnData object
#'
#' @import reticulate
.var2feature_metadata <- function(var_pd) {
  var_df <- .regularise_df(var_pd, drop_single_values=FALSE, drop_na_values=TRUE)
  colnames(var_df) <- sub("dispersions_norm", "mvp.dispersion.scaled", colnames(var_df))
  colnames(var_df) <- sub("dispersions", "mvp.dispersion", colnames(var_df))
  colnames(var_df) <- sub("means", "mvp.mean", colnames(var_df))
  colnames(var_df) <- sub("highly_variable", "highly.variable", colnames(var_df))
  return(var_df)
}

In [ ]:
obs_df <- .obs2metadata(adata$obs)
var_df <- .var2feature_metadata(adata$var)
X <- t(adata$X)
colnames(X) <- rownames(obs_df)
rownames(X) <- rownames(var_df)
X

In [ ]:
rownames(obs_df)

In [ ]:
rownames(var_df)

In [ ]:
obs_df

In [ ]:
x_t <- Matrix::t(adata$X)
x_t

In [ ]:
adata_Seurat$RNA$counts

In [ ]:
adata_Seurat$RNA$counts

In [ ]:
Convert(path, "h5seurat", overwrite = TRUE)

In [ ]:
srat <- LoadH5Seurat(paste0(tools::file_path_sans_ext(path), ".h5seurat"), meta.data = FALSE, misc = FALSE)

In [ ]:
library(Seurat)
library(SingleCellExperiment)
library(SeuratDisk)
library(SeuratData)
input = "/ps/ai-ready/data/error/tung.rds"
robj <- readRDS(input)
robj

In [ ]:
logcounts(robj)

In [ ]:
'logcounts'  %in% names(robj)

In [ ]:
srat <- as.Seurat(robj, slot = "counts")

In [ ]:
# path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.Robj"
path <- "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat"

In [ ]:
srat <- LoadSeurat(path)
srat

In [ ]:
adata <- SeuratToAnndata(srat)
adata

In [ ]:
adata$X

In [ ]:
obj <- srat
default_assay <- 'RNA'
if(paste0("nCount_", default_assay) %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) ==paste0("nCount_", default_assay)] <-"n_counts"
if(paste0("nFeature_", default_assay) %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) ==paste0("nFeature_", default_assay)] <-"n_genes"
if("percent.mt" %in% names(obj@meta.data)) names(obj@meta.data)[names(obj@meta.data) =="percent.mt"] <-"percent_mito"
obj@meta.data

In [ ]:
adata$layers['scale.data']

In [ ]:
srat[['RNA']]@counts

In [ ]:
mat <- Seurat::GetAssayData(object=srat, assay='RNA', layer='X')

In [ ]:
default_assay <- DefaultAssay(srat)
default_assay

In [ ]:
adata <- LoadAnndata("/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad")
adata

In [ ]:
adata$obsm['X_pca']

In [ ]:
Embeddings(object = srat, reduction = "pca")

In [ ]:
max(srat[['RNA']]@counts)

In [ ]:
min(srat[['RNA']]@counts)

In [ ]:
doublet_annnotation <- AnnotateDroplet(srat[[default_assay]]@counts)

In [ ]:
doublet_annnotation$doublet_score

In [ ]:
srat[["Doublet_score"]] <- doublet_annnotation$doublet_score
srat[["Doublet_class"]] <- doublet_annnotation$doublet_class

In [ ]:
data_mat = srat[['RNA']]@counts

In [ ]:
# -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
srat[["Doublet_score"]] <- doublet_score

In [ ]:
srat[["Doublet_class"]] <- doublet_class

In [ ]:
srat[[]]

In [ ]:
dim(srat[['RNA']]@counts)
dim(srat)
dim(srat@meta.data)

In [ ]:
if(!paste0("nCount_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nCount_", default_assay)]] <- colSums(x = srat[[default_assay]], slot = "counts")  # nCount of the default assay
if(!paste0("nFeature_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nFeature_", default_assay)]] <- colSums(x = GetAssayData(object = srat[[default_assay]], slot = "counts") > 0)  # nFeature of the default assay

In [ ]:
head(srat@meta.data)

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)

In [ ]:
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat1@meta.data

In [ ]:
adata_path <- Convert('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat', dest = "h5ad" , overwrite = TRUE)
adata_path

In [ ]:
# Calculate the percentage of mitocondrial per cell and add to the metadata.
if(! "percent.mt" %in% names(x = srat[[]])) srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-")
# Calculate the proportion gene expression that comes from ribosomal proteins.
if(! "percent.rb" %in% names(x = srat[[]])) srat[["percent.rb"]] <- PercentageFeatureSet(srat, pattern = "^RP[SL]")

# Percentage hemoglobin genes - includes all genes starting with HB except HBP.
if(! "percent.hb" %in% names(x = srat[[]])) srat[["percent.hb"]] <- PercentageFeatureSet(srat, pattern = "^HB[^(P)]")
if(! "percent.plat" %in% names(x = srat[[]])) srat[["percent.plat"]] <- PercentageFeatureSet(srat, pattern = "PECAM1|PF4")
head(srat@meta.data)

In [ ]:
# Add the doublet annotation
doublets <- read.table(here::here('api/tools/qc/scrublet_calls.tsv'), header = F, row.names = 1)
colnames(doublets) <- c("Doublet_score", "Is_doublet")
srat <- AddMetaData(srat, doublets)
# srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
head(srat@meta.data)

In [ ]:
srat@meta.data

In [ ]:
names(srat@meta.data)[names(srat@meta.data) =="nCount_RNA"] <-"n_counts"
srat@meta.data

In [ ]:
names(srat@meta.data)

In [ ]:
nFeature_min=200
nFeature_max=0
percent_mt_max=5
percent_rb_min=0
srat <- subset(srat, subset = paste0("nFeature_", default_assay) > nFeature_min & percent.mt < percent_mt_max)
if(nFeature_max != 0) srat <- subset(srat, subset = paste0("nFeature_", default_assay) < nFeature_max)
if(percent_rb_min != 0)  srat <- subset(srat, subset = percent.rb > percent_rb_min)

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True' | is.na(Is_doublet))

In [ ]:
#' Regularise dataframe
#'
#' This function checks if certain columns of a dataframe is of a single value
#' and drop them if required
#'
#' @param df Input data frame, usually cell metadata table (data.frame-like
#'   object)
#' @param drop_single_values Drop columns with only a single value (logical)
#'
#' @return Dataframe
.regularise_df <- function(df, drop_single_values=FALSE, drop_na_values=TRUE) {
  if (ncol(df) == 0) df[["name"]] <- rownames(df)
  if (drop_single_values) {
    k_singular <- sapply(df, function(x) length(unique(x)) == 1)
    if (sum(k_singular) > 0) {
      warning(
        paste("Dropping single category variables:"),
        paste(colnames(df)[k_singular], collapse = ", ")
      )
    }
    df <- df[, !k_singular, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
 if (drop_na_values) {
    k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
    if (sum(k_na) > 0) {
      warning(
        paste("Dropping NA category variables:"),
        paste(colnames(df)[k_na], collapse = ", ")
      )
    }
    df <- df[, !k_na, drop = F]
    if (ncol(df) == 0) df[["name"]] <- rownames(df)
  }
  return(df)
}

In [ ]:
output_path_rds <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.rds'
saveRDS(srat, output_path_rds)
srat1 <- LoadSeurat(output_path_rds)
srat1

In [ ]:
k_na <- sapply(srat@meta.data, function(x) sum(is.na(x))==length(x))
k_na
sum(k_na)

In [ ]:
srat_qc@meta.data %>% is.na()

In [ ]:
k_singular <- sapply(srat_qc@meta.data, function(x) length(unique(x)) == 1)
k_singular

In [ ]:
srat_qc@meta.data

In [ ]:
srat@meta.data <- .regularise_df(srat@meta.data, drop_single_values=FALSE, drop_na_values=TRUE)
srat@meta.data

In [ ]:
srat_qc@meta.data <- .regularise_df(srat_qc@meta.data, drop_single_values = FALSE)
srat_qc@meta.data

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_3 <- LoadH5Seurat(output_path)
srat_3

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat1, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_2 <- LoadH5Seurat(output_path)
srat_2

In [ ]:
srat1[["meta.data"]]

In [ ]:
srat1@meta.data

In [ ]:
devtools::install_github("cellgeni/sceasy")

In [ ]:
sceasy::convertFormat(srat1, from="seurat", to="anndata",
                       outFile='/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')

In [ ]:
adata <- LoadAnndata('/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad')
adata

In [ ]:
adata$obs

In [ ]:
adata$X

In [ ]:
srat_qc <- CreateSeuratObject(counts=srat[['RNA']]@counts, meta.data=srat@meta.data, project = Project(srat))
srat_qc

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat_qc, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat_qc_1 <- LoadH5Seurat(output_path)
srat_qc_1

In [ ]:
output_path <- '/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5Seurat'
SaveH5Seurat(srat, filename = output_path, overwrite = TRUE, verbose = TRUE)
srat1 <- LoadH5Seurat(output_path)
srat1

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')

In [ ]:
srat[["meta.data"]]

In [ ]:
Cells(srat)

In [ ]:
srat <- NormalizeData(srat, normalization.method = "LogNormalize", scale.factor = 10000)
srat <- FindVariableFeatures(srat, selection.method = "vst")
srat <- ScaleData(srat, features = rownames(srat))
srat
dim(srat)
dim(srat@meta.data)

In [ ]:
srat[['Doublet_score']][sapply(srat[['Doublet_score']], is.na)] <- 0
srat[['Doublet_score']]

In [ ]:
srat[['Doublet_score']] <- !is.na(srat[['Doublet_score']])


In [ ]:
# srat <- subset(srat, subset = percent.rb >= 0)
srat <- subset(srat, subset = is.na(Is_doublet))
srat

In [ ]:
srat[['Is_doublet']] <- !is.na(srat[['Is_doublet']])

In [ ]:
srat <- subset(srat, subset = Is_doublet != 'True')
srat

In [ ]:
srat@meta.data

In [ ]:
'nCount_RNA' %in% names(srat@meta.data)

In [ ]:
result <- GetMetadataFromSeurat(path)
result

In [ ]:
suffix <- tolower(get_suffix(path))
suffix

In [ ]:
seurat_object <- LoadSeurat(path)
seurat_object

In [ ]:
'umap' %in% names(seurat_object@reductions)

In [ ]:
genes <- rownames(seurat_object)
typeof(genes)
genes

In [ ]:
cells <- Cells(seurat_object)
typeof(cells)
cells

In [ ]:
pca <- Embeddings(object = seurat_object, reduction = "pca")
typeof(pca)
pca

In [ ]:
name <- load(path)
seurat_object <- UpdateSeuratObject(get(name))

In [ ]:
seurat_object

In [ ]:
features <- seurat_object[['RNA']]@var.features
typeof(features)
features

In [ ]:
#获取当前用的Idents
Idents(seurat_object)
levels(seurat_object)

In [ ]:
#总数目
nCells <- ncol(seurat_object)
nGenes <- nrow(seurat_object)

nCells
nGenes

In [ ]:
dim=1:10
class(dim)

In [ ]:
x <- 1:10
x

In [ ]:
class(x)

In [ ]:
devtools::install_github('JiekaiLab/dior')

In [ ]:
source("../tools/benchmarks/clustering/seurat.R")
path = "/ps/ai-ready/data/Clustering/Tabula_Muris/droplet_Bladder_seurat_tiss.h5ad"

In [ ]:
clustering <- function(path, labels, dims=1:10){ # labels: column name of labels in srat@meta.data
    srat <- tryCatch(
        LoadSeurat(path),
        error = function(e) {
            stop("The file format is not supported.")
            print(e)
        }
    )

    srat <- FindVariableFeatures(srat, selection.method = "vst")
    srat <- ScaleData(srat, features = rownames(srat))

    # PCA
    # srat <- RunPCA(srat, features = VariableFeatures(srat), ndims.print = 6:10, nfeatures.print = 10)
    srat <- RunPCA(srat, features = VariableFeatures(srat))

    srat <- FindNeighbors(srat, dims=dims)
    srat <- FindClusters(srat, resolution = 0.5)
    srat

    # TSNE
    srat <- RunTSNE(srat, dims=dims)
    # UMAP
    srat <- RunUMAP(srat, dims=dims)

    umap <- Embeddings(object = srat, reduction = "umap")

    list(labels=as.list(srat@meta.data[labels]), labels_pred=as.list(srat@meta.data["seurat_clusters"]), umap=umap)
}

In [ ]:
result <- clustering(path, 'cluster.ids')
result

In [ ]:
result